In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.express as px
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2024-04-19 21:31:16.048903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 21:31:16.049060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 21:31:16.225723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd
column_names = ["target", "id", "date", "flag", "user", "text"]
df = pd.read_csv("/kaggle/input/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv",encoding="ISO-8859-1", names=column_names)
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
data = df[["target","text"]]
data['target'][data['target']==4]=1
data['text']=data['text'].str.lower()

In [4]:
stopwords_list = stopwords.words('english')

In [5]:
STOPWORDS = set(stopwords_list)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))

In [6]:
def cleaning_email(data):
    return re.sub('@[^\s]+', ' ', data)

data['text']= data['text'].apply(lambda x: cleaning_email(x))
data['text'].head()

0      http://twitpic.com/2y1zl - awww, that's bumm...
1    upset can't update facebook texting it... migh...
2      dived many times ball. managed save 50% rest...
3                     whole body feels itchy like fire
4      no, behaving all. i'm mad. here? can't see t...
Name: text, dtype: object

In [7]:
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)
data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))

In [8]:
english_punctuations = string.punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', english_punctuations)
    return text.translate(translator)
data['text'] = data['text'].apply(lambda text: cleaning_punctuations(text))

In [9]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))

In [10]:
data['text'] = data['text'].astype(str)

In [11]:
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)


In [12]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatizer_on_text(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]
data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [13]:
texts=data.text
labels=data.target
max_len = 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(texts)
sequences = tok.texts_to_sequences(texts)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [14]:
vocab_size = len(tok.word_index)
if vocab_size > 2000:
    vocab_size = 2000

sequence_length = max_len

print("Vocabulary Size:", vocab_size)
print("Sequence Length:", sequence_length)

Vocabulary Size: 2000
Sequence Length: 500


In [15]:
from transformers import pipeline 
num_classes = 2 

# Tokenize text
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=sequence_length, padding='post')

labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
batch_size = 64
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).shuffle(buffer_size=1024)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

translation_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Create a function to translate text to English
def translate_to_english(text, src_lang):
    translated_text = translation_pipe(text, src_lang=src_lang, tgt_lang='en')
    return translated_text[0]['translation_text']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [16]:
class TransformerEncoderSentiment(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dense_dim, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.dropout_1 = tf.keras.layers.Dropout(dropout)
        self.dropout_2 = tf.keras.layers.Dropout(dropout)
    
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype=tf.int32)
        else:
            padding_mask = None
        
        attention_output = self.attention(inputs, inputs, attention_mask=padding_mask)
        proj_input = self.layernorm_1(inputs + self.dropout_1(attention_output))
        
        proj_output = self.dense_proj(proj_input)
        encoder_output = self.layernorm_2(proj_input + self.dropout_2(proj_output))
        
        return encoder_output

# Define model architecture
embed_dim = 128
num_heads = 4
dense_dim = 256
dropout_rate = 0.1

inputs = tf.keras.layers.Input(shape=(sequence_length,), dtype=tf.int32)
embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerEncoderSentiment(embed_dim, num_heads, dense_dim, dropout_rate)
x = transformer_block(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 500, 128)       │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_sentiment   │ (None, 500, 128)       │       330,240 │
│ (TransformerEncoderSentiment)   │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 586,498 (2.24 MB)

 Trainable params: 586,498 (2.24 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
epochs = 5

# Train the model
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

# Evaluate the model
loss, accuracy = model.evaluate(val_ds)
print("Validation Accuracy:", accuracy)


Epoch 1/5
    2/20000 ━━━━━━━━━━━━━━━━━━━━ 27:30 83ms/step - accuracy: 0.5039 - loss: 1.5856  

I0000 00:00:1713563231.774268      85 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713563231.791475      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1377s 68ms/step - accuracy: 0.7412 - loss: 0.5193 - val_accuracy: 0.7603 - val_loss: 0.4887
Epoch 2/5
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1367s 68ms/step - accuracy: 0.7600 - loss: 0.4893 - val_accuracy: 0.7614 - val_loss: 0.4870
Epoch 3/5
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1370s 68ms/step - accuracy: 0.7634 - loss: 0.4843 - val_accuracy: 0.7639 - val_loss: 0.4829
Epoch 4/5
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1367s 68ms/step - accuracy: 0.7645 - loss: 0.4813 - val_accuracy: 0.7637 - val_loss: 0.4829
Epoch 5/5
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1367s 68ms/step - accuracy: 0.7664 - loss: 0.4788 - val_accuracy: 0.7640 - val_loss: 0.4825
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 112s 22ms/step - accuracy: 0.7646 - loss: 0.4818
Validation Accuracy: 0.7640030980110168


In [18]:
import os

# Save the model using tf.saved_model.save()
tf.saved_model.save(model, "sentiment_analysis_model")

In [19]:
tokenizer_config = tokenizer.to_json()
with open("tokenizer_config.json", "w") as tokenizer_file:
    tokenizer_file.write(tokenizer_config)

# Define a function to predict sentiment for text in any language
def predict_sentiment(text, src_lang):
    # Translate text to English
    english_text = translate_to_english(text, src_lang)
    
    # Preprocess the text
    sequence = tokenizer.texts_to_sequences([english_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length, padding='post')
    
    # Make prediction
    predicted_probabilities = model.predict(padded_sequence)
    predicted_class = np.argmax(predicted_probabilities)
    
    # Map predicted class to sentiment label
    sentiment_label = "Positive" if predicted_class == 1 else "Negative"
    
    return sentiment_label, predicted_probabilities

# Define a sample text in different languages
sample_texts = {
    'fr': 'Je suis heureux.',
    'es': 'Estoy feliz.',
    'de': 'Ich bin glücklich.',
    'it': 'Sono felice.',
    'zh-cn': '我今天很伤心.'
}

# Predict sentiment for each sample text
for lang, text in sample_texts.items():
    sentiment, probabilities = predict_sentiment(text, lang)
    print(f"Input ({lang}): {text}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Predicted Probabilities: {probabilities}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 851ms/step
Input (fr): Je suis heureux.
Predicted Sentiment: Positive
Predicted Probabilities: [[0.11296368 0.88703626]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input (es): Estoy feliz.
Predicted Sentiment: Positive
Predicted Probabilities: [[0.11296368 0.88703626]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input (de): Ich bin glücklich.
Predicted Sentiment: Positive
Predicted Probabilities: [[0.11296368 0.88703626]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input (it): Sono felice.
Predicted Sentiment: Positive
Predicted Probabilities: [[0.11296368 0.88703626]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input (zh-cn): 我今天很伤心.
Predicted Sentiment: Negative
Predicted Probabilities: [[0.95004827 0.04995171]]



In [ ]:
# Zip the model files
!zip -r sentiment_analysis_model.zip sentiment_analysis_model tokenizer_config.json

# Move the zipped file to the output directory
import shutil
shutil.move("sentiment_analysis_model.zip", "/kaggle/working/sentiment_analysis_model.zip")

In [ ]:
# Define a sample English text for prediction
sample_text = "I loved the movie! It was fantastic."

# Preprocess the sample text
sample_sequence = tokenizer.texts_to_sequences([sample_text])
sample_padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sample_sequence, maxlen=sequence_length, padding='post')

# Make the prediction
predicted_probabilities = model.predict(sample_padded_sequence)
predicted_class = np.argmax(predicted_probabilities)

# Map the predicted class to sentiment label
sentiment_label = "Positive" if predicted_class == 1 else "Negative"

print("Sample Text:", sample_text)
print("Predicted Sentiment:", sentiment_label)
print("Predicted Probabilities:", predicted_probabilities)


In [ ]:
import tkinter as tk
from tkinter import ttk
from transformers import pipeline

# Load translation pipeline
translation_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define language codes and names
languages = {
    'af': 'Afrikaans',
    'ar': 'Arabic',
    'bg': 'Bulgarian',
    'bn': 'Bengali',
    'ca': 'Catalan',
    'cs': 'Czech',
    'cy': 'Welsh',
    'da': 'Danish',
    'de': 'German',
    'el': 'Greek',
    'en': 'English',
    'es': 'Spanish',
    'et': 'Estonian',
    'fa': 'Persian',
    'fi': 'Finnish',
    'fr': 'French',
    'gu': 'Gujarati',
    'he': 'Hebrew',
    'hi': 'Hindi',
    'hr': 'Croatian',
    'hu': 'Hungarian',
    'id': 'Indonesian',
    'it': 'Italian',
    'ja': 'Japanese',
    'kn': 'Kannada',
    'ko': 'Korean',
    'lt': 'Lithuanian',
    'lv': 'Latvian',
    'mk': 'Macedonian',
    'ml': 'Malayalam',
    'mr': 'Marathi',
    'ne': 'Nepali',
    'nl': 'Dutch',
    'no': 'Norwegian',
    'pa': 'Punjabi',
    'pl': 'Polish',
    'pt': 'Portuguese',
    'ro': 'Romanian',
    'ru': 'Russian',
    'si': 'Sinhala',
    'sk': 'Slovak',
    'sl': 'Slovenian',
    'so': 'Somali',
    'sq': 'Albanian',
    'sv': 'Swedish',
    'sw': 'Swahili',
    'ta': 'Tamil',
    'te': 'Telugu',
    'th': 'Thai',
    'tl': 'Tagalog',
    'tr': 'Turkish',
    'uk': 'Ukrainian',
    'ur': 'Urdu',
    'vi': 'Vietnamese',
    'zh-cn': 'Chinese (Simplified)',
    'zh-tw': 'Chinese (Traditional)'
}

# Function to translate text to English
def translate_to_english(text, src_lang):
    translated_text = translation_pipe(text, src_lang=src_lang, tgt_lang='en')
    return translated_text[0]['translation_text']

# Function to perform sentiment analysis
def predict_sentiment(text, src_lang):
    # Translate text to English
    english_text = translate_to_english(text, src_lang)
    
    # Preprocess the text
    sequence = tokenizer.texts_to_sequences([english_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length, padding='post')
    
    # Make prediction
    predicted_probabilities = model.predict(padded_sequence)
    predicted_class = np.argmax(predicted_probabilities)
    
    # Map predicted class to sentiment label
    sentiment_label = "Positive" if predicted_class == 1 else "Negative"
    
    return sentiment_label, predicted_probabilities

# Function to handle button click event
def analyze_sentiment():
    input_text = text_entry.get("1.0", "end-1c")
    src_lang = lang_combobox.get()
    
    sentiment, probabilities = predict_sentiment(input_text, src_lang)
    
    result_text.set(f"Sentiment: {sentiment}\nProbabilities: {probabilities}")

# Create main window
root = tk.Tk()
root.title("Sentiment Analysis")

# Create input label and text entry
input_label = ttk.Label(root, text="Enter text:")
input_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

text_entry = tk.Text(root, height=5, width=40)
text_entry.grid(row=0, column=1, padx=10, pady=5)

# Create language selection combobox
lang_label = ttk.Label(root, text="Select language:")
lang_label.grid(row=1, column=0, padx=10, pady=5, sticky="w")

language_codes = list(languages.keys())
language_names = list(languages.values())
lang_combobox = ttk.Combobox(root, values=language_names, state="readonly")
lang_combobox.current(0)  # Set default language
lang_combobox.grid(row=1, column=1, padx=10, pady=5)

# Create analyze button
analyze_button = ttk.Button(root, text="Analyze Sentiment", command=analyze_sentiment)
analyze_button.grid(row=2, column=0, columnspan=2, padx=10, pady=5)

# Create label for displaying result
result_text = tk.StringVar()
result_label = ttk.Label(root, textvariable=result_text)
result_label.grid(row=3, column=0, columnspan=2, padx=10, pady=5)

root.mainloop()
